In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as dsets
from torchvision import models
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter

#writer = SummaryWriter('runs/thesis06_miniIN5w1s_5step')

from tqdm import tqdm

from flows import PlanarFlow
from utils import Binarize
from codes import Linear_flipout, Flatten, count_parameters, EfficientNet

from torchmeta.datasets import MiniImagenet
from torchmeta.transforms import Categorical, ClassSplitter, Rotation
from torchvision.transforms import Compose, Resize, ToTensor
from torchmeta.utils.data import BatchMetaDataLoader


#from __future__ import print_function
import argparse
import cv2
import matplotlib.pyplot as plt

import os
cur_dir = "C:/Users/KJH/OneDrive - skku.edu/KJH/Projects/2019winter_research"
#cur_dir = "C:/Users/KJH-Laptop/OneDrive - skku.edu/KJH/Projects/2019winter_research/"
os.chdir(cur_dir)
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import time
import copy
import random as rd

device = torch.device('cuda')

class net(nn.Module):
    def __init__(self, num_classes):
        super(net, self).__init__()
        self.input_dim = [3, 84, 84]
        self.num_classes = num_classes
        
        self.ctx = torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True)
        self.ctx.classifier = nn.Identity()
        
        self.layer = nn.LSTM(input_size=1280, hidden_size=1280, num_layers=5, bias=True, batch_first = True).to(device)
        self.dec_mu = nn.ModuleList()
        self.dec_logvar = nn.ModuleList()
        
        self.num_params = [[200 * 200, 200], [200 * 200, 200], [200 * self.num_classes, self.num_classes]]
        
        for layer_size in self.num_params:
            self.dec_mu.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
            
            self.dec_logvar.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
        
        self.encoder = nn.Sequential(
            torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True),
            Flatten(),            
            nn.Linear(1280, 200, bias = False),
            nn.ELU()
        )
        self.encoder[0].classifier = nn.Identity()
            
            
    def forward(self, input_train, label_train, input_test, label_test, adapt_lr, adapt_step = 1):
        ctx = self.ctx(input_train).view(input_train.shape[0], self.num_classes, 256)
        h = torch.stack([ctx[torch.where(label_train == x)].mean(dim = 0) for x in range(self.num_classes)], dim = 1)
        h = h.view(self.num_classes, 1, -1)
        
        params = nn.ParameterList()
        param_vals = []
        kld = torch.tensor(0., device=device)
        c = torch.empty([5, 1, 1280], requires_grad = False, device = device).fill_(0)
        x_train_init = self.encoder(input_train)
        x_train = x_train_init

        for ind, (dec_mu, dec_logvar) in enumerate(zip(self.dec_mu, self.dec_logvar)):
            x = torch.empty([1, 1, 1280], requires_grad=False, device = device).normal_(0, 1)
            f, (h, c) = self.layer(x, (h, c))
            
            mu = dec_mu(f.view(1, -1)).squeeze()
            logvar = dec_logvar(f.view(1, -1)).squeeze()
            params.append(nn.Parameter(torch.stack((mu, logvar), dim = 0), requires_grad = True))
        
        optimizer = optim.SGD(params, lr = adapt_lr)
        
        for step in range(adapt_step):
            x_train = x_train_init
            kld = torch.tensor(0., device=device)
            for ind, param in enumerate(params):
                param.retain_grad()

                weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_mu = param[0, self.num_params[ind][0]:]

                weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_logvar = param[1, self.num_params[ind][0]:]

                weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
                bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
                in_sign = torch.empty(x_train.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
                out_sign = torch.empty([x_train.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

                x_train = torch.mm(x_train, weight_mu) + torch.mm(in_sign * x_train, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
                x_train += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
                x_train = F.elu(x_train)

                kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2
                
            if step < adapt_step:
                optimizer.zero_grad()
                loss = F.cross_entropy(x_train, label_train) + 1e-6 * kld
                loss.backward(retain_graph = True)
                optimizer.step()
                
                
        x_test_init = self.encoder(input_test)
        x_test = x_test_init
        kld = torch.tensor(0., device=device)
        for ind, param in enumerate(params):
            param.retain_grad()

            weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_mu = param[0, self.num_params[ind][0]:]

            weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
            bias_logvar = param[1, self.num_params[ind][0]:]

            weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
            bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
            in_sign = torch.empty(x_test.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
            out_sign = torch.empty([x_test.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

            x_test = torch.mm(x_test, weight_mu) + torch.mm(in_sign * x_test, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
            x_test += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
            x_test = F.elu(x_test)

            kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2

        return x_test, kld

In [2]:
batch_size = 16
meta_trainset = MiniImagenet('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(84), ToTensor()]), #Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_train=True,
                   download=True)
meta_trainset = ClassSplitter(meta_trainset, shuffle=True, num_train_per_class=1, num_test_per_class=15)
meta_trainloader = BatchMetaDataLoader(meta_trainset, batch_size=batch_size, num_workers=0)

model = net(5).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


  0) loss = 1.603300, kld = 0.000912, acc = 0.220833, time = 14.028 sec
 10) loss = 1.557080, kld = 0.000692, acc = 0.330833, time = 13.184 sec
 20) loss = 1.496808, kld = 0.000533, acc = 0.365000, time = 13.599 sec
 30) loss = 1.470685, kld = 0.000424, acc = 0.369167, time = 13.583 sec
 40) loss = 1.459826, kld = 0.000344, acc = 0.372500, time = 13.456 sec
 50) loss = 1.436125, kld = 0.000280, acc = 0.372500, time = 13.015 sec
 60) loss = 1.493082, kld = 0.000234, acc = 0.353333, time = 13.309 sec
 70) loss = 1.470079, kld = 0.000202, acc = 0.363333, time = 13.154 sec
 80) loss = 1.501140, kld = 0.000174, acc = 0.347500, time = 13.257 sec
 90) loss = 1.513136, kld = 0.000154, acc = 0.344167, time = 13.470 sec
100) loss = 1.433737, kld = 0.000139, acc = 0.365000, time = 13.180 sec
110) loss = 1.403915, kld = 0.000124, acc = 0.390000, time = 13.177 sec
120) loss = 1.439921, kld = 0.000117, acc = 0.401667, time = 13.621 sec
130) loss = 1.452853, kld = 0.000110, acc = 0.369167, time = 13.

In [3]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 1.347534, kld = 0.000078, acc = 0.439167, time = 13.577 sec
 10) loss = 1.399070, kld = 0.000079, acc = 0.395833, time = 13.079 sec
 20) loss = 1.446461, kld = 0.000077, acc = 0.378333, time = 13.212 sec
 30) loss = 1.383013, kld = 0.000079, acc = 0.405000, time = 13.414 sec
 40) loss = 1.447785, kld = 0.000079, acc = 0.375833, time = 13.253 sec
 50) loss = 1.376030, kld = 0.000078, acc = 0.421667, time = 13.135 sec
 60) loss = 1.403178, kld = 0.000082, acc = 0.404167, time = 13.310 sec
 70) loss = 1.361057, kld = 0.000081, acc = 0.414167, time = 13.316 sec
 80) loss = 1.389949, kld = 0.000080, acc = 0.414167, time = 13.335 sec
 90) loss = 1.344845, kld = 0.000081, acc = 0.439167, time = 13.230 sec
100) loss = 1.357444, kld = 0.000078, acc = 0.426667, time = 13.005 sec
110) loss = 1.322672, kld = 0.000079, acc = 0.445833, time = 13.269 sec
120) loss = 1.351838, kld = 0.000080, acc = 0.451667, time = 13.220 sec
130) loss = 1.374991, kld = 0.000082, acc = 0.403333, time = 13.

In [4]:
optimizer = optim.Adam(model.parameters(), lr=1e-7)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 75
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 1.429130, kld = 0.000080, acc = 0.397500, time = 13.177 sec
 10) loss = 1.370641, kld = 0.000077, acc = 0.408333, time = 13.360 sec
 20) loss = 1.383376, kld = 0.000081, acc = 0.422500, time = 13.044 sec
 30) loss = 1.419267, kld = 0.000079, acc = 0.395833, time = 12.979 sec
 40) loss = 1.379146, kld = 0.000079, acc = 0.403333, time = 13.215 sec
 50) loss = 1.410678, kld = 0.000077, acc = 0.400833, time = 13.612 sec
 60) loss = 1.390218, kld = 0.000082, acc = 0.415833, time = 13.391 sec
 70) loss = 1.335163, kld = 0.000078, acc = 0.432500, time = 13.117 sec
 80) loss = 1.352390, kld = 0.000077, acc = 0.419167, time = 13.564 sec
 90) loss = 1.452623, kld = 0.000078, acc = 0.383333, time = 13.293 sec
100) loss = 1.451398, kld = 0.000078, acc = 0.365000, time = 13.393 sec
110) loss = 1.344446, kld = 0.000079, acc = 0.427500, time = 13.466 sec
120) loss = 1.330144, kld = 0.000079, acc = 0.445833, time = 13.361 sec
130) loss = 1.300217, kld = 0.000079, acc = 0.452500, time = 13.

In [ ]:
meta_testset  = MiniImagenet('./data/',
                   # Number of ways
                   num_classes_per_task=5,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=5),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_test=True,
                   download=True)
meta_testset = ClassSplitter(meta_testset, shuffle=True, num_train_per_class=1, num_test_per_class=15)
meta_testloader = BatchMetaDataLoader(meta_testset, batch_size=batch_size, num_workers=0)

tot_loss = torch.tensor(0., device=device)
tot_acc = torch.tensor(0., device=device)
tot_reg = torch.tensor(0., device=device)

for batch_idx, meta_test_batch in zip(range(num_batches), meta_testloader):
    start = time.time()
    train_inputs, train_targets = [x.to(device) for x in meta_test_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_test_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, 1, 5)
        with torch.no_grad():            
            loss = criterion(pred, test_target)
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld

    tot_loss += cum_loss / batch_size
    tot_acc += accuracy  / (batch_size * 75)
    tot_reg += reg / batch_size
        
tot_loss /= num_batches
tot_acc /= num_batches
tot_reg /= num_batches
        
print("loss = %f, test_kld = %f, meta_test_acc = %f, time = %.3f sec" %(tot_loss, tot_reg, tot_acc, time.time() - start))

In [ ]:
torch.save(model.state_dict(), "./save/thesis06_miniIN5w1s_5step")